### Neuroevolution:
- Bild -> preprocessing -> CNN -> MLP -> Eingabe (NES controller)
- CNN und MLP basieren auf pytorch
- CNN und MLP werden gemeinsam mutiert, die mutation vom agenten der am weitesten kam wird übernommen
- Distanz vom Start wird (fürs erste) der einzige reward sein

In [ ]:
from src.non_reinforcement.neuroevolution.neuroevolution_agent import NeuroevolutionAgent
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet

import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT
from gym.wrappers import RecordVideo
import os

# For saving video
video_dir = "../media/videos"
os.makedirs(video_dir, exist_ok=True)

# Create environment and start recording video
env = JoypadSpace(gym_super_mario_bros.make('SuperMarioBros-v0'), COMPLEX_MOVEMENT)
env = RecordVideo(env, video_dir, episode_trigger=lambda x: True)

# Create agent
model = NeuroevolutionNet(num_actions=env.action_space.n)
agent = NeuroevolutionAgent(model)

# Evaluate agent
fitness = agent.evaluate(env)
env.close()

print(f"Fitness: {fitness}")